In [1]:
import pandas as pd
import dtale
import FundamentalAnalysis as fa
import yfinance as yf
import finviz
from datetime import date
from datetime import datetime
from dateutil import relativedelta

In [2]:
def insider_date(symbol):
    for i in finviz.get_insider(symbol):
        tradedate = [i['Date'] + ' 2021']
        insiderdf = pd.DataFrame.from_dict(i)
        tradedate = pd.to_datetime(tradedate, format = '%b %d %Y')
    return tradedate[0]

In [3]:
def get_finviz(symbol, date):
    #Get insider trading information and fix the date
    #format in the finviz api so that the amount of time that has passed since purchase can be determined
    
    finStock = finviz.get_insider(symbol)
    finStock = pd.DataFrame.from_dict(finStock)
    dtFinStock = pd.to_datetime(finStock['Date'], format = '%b %d')
    finStock['Date'] = dtFinStock
    today = pd.to_datetime(date.today())
    thisYear = finStock['Date'].loc[finStock['Date'].dt.strftime('%m-%d') <= today.strftime('%m-%d')]
    updatedDates = [None for x in range(finStock['Date'].size)]
    count = 0
    for date in thisYear:
        thisYear = date.replace(2021)
        updatedDates[count] = thisYear
        count = count + 1


    lastYear = finStock['Date'].loc[finStock['Date'].dt.strftime('%m-%d') > today.strftime('%m-%d')]
    for date in lastYear:
        lastYear = date.replace(2020)
        updatedDates[count] = lastYear
        count = count + 1


    finStock['Date'] = updatedDates
    finStock.set_index('Date', inplace=True)
    return finStock

In [4]:
def six_month_insider(symbol):
    #Determine the total value traded by insiders in the last 6 months
    try:
        finDf = get_finviz(symbol, date)
        idxSixMonth = finDf[datetime.today() : datetime.today() - relativedelta.relativedelta(months=6)]
        sixMonthValue = idxSixMonth['Value ($)']
        sixMonthValue = pd.to_numeric(sixMonthValue.str.replace(',',''), errors='coerce')
        transaction = idxSixMonth['Transaction']
        sixMonthValue[transaction == 'Sale'] = -sixMonthValue[transaction == 'Sale'] 
        sixMonthValue[transaction == 'Option Exercise'] = -sixMonthValue[transaction == 'Option Exercise'] 
        sixMonthSum = sixMonthValue.sum(axis = 0)
    except:
        sixMonthSum = 0
    return sixMonthSum

In [5]:
def one_month_insider(symbol):
    #Determine the total value traded by insiders in the last 1 month
    try:
        finDf = get_finviz(symbol, date)
        idxOneMonth = finDf[datetime.today() : datetime.today() - relativedelta.relativedelta(months=1)]
        oneMonthValue = idxOneMonth['Value ($)']
        oneMonthValue = pd.to_numeric(oneMonthValue.str.replace(',',''), errors='coerce')
        transaction = idxOneMonth['Transaction']
        oneMonthValue[transaction == 'Sale'] = -oneMonthValue[transaction == 'Sale'] 
        oneMonthValue[transaction == 'Option Exercise'] = -oneMonthValue[transaction == 'Option Exercise'] 
        oneMonthSum = oneMonthValue.sum(axis = 0)
    except:
        oneMonthSum = 0
    return oneMonthSum

In [6]:
def one_week_insider(symbol):
    #Determine the total value traded by insiders in the last 1 week
    try:
        finDf = get_finviz(symbol, date)
        idxOneWeek = finDf[datetime.today() : datetime.today() - relativedelta.relativedelta(week=1)]
        oneWeekValue = idxOneWeek['Value ($)']
        oneWeekhValue = pd.to_numeric(oneWeekValue.str.replace(',',''), errors='coerce')
        transaction = idxOneWeek['Transaction']
        oneWeekValue[transaction == 'Sale'] = -oneWeekValue[transaction == 'Sale'] 
        oneWeekValue[transaction == 'Option Exercise'] = -oneWeekValue[transaction == 'Option Exercise'] 
        oneWeekSum = oneWeekValue.sum(axis = 0)
    except:
        oneWeekSum = 0
    return oneWeekSum

In [7]:
def make_fundamental_sheet(df):
    #Convert the fundamnetal data to numerical so color formatting can be applied
    dfDf = pd.DataFrame()
    for i in range(len(df)):
        dfNum = pd.to_numeric(df.iloc[i], errors='coerce')
        dfNum = pd.DataFrame(dfNum)
        dfNum = dfNum.T
        dfDf = pd.concat([dfDf, dfNum])
    return dfDf

In [8]:
def make_percent_change(idx,df):
    #Calculate the percent change for the fundamental dataframe
    pctChange = df.iloc[idx]
    pctChange = pctChange.pct_change(periods = -1)
    pctChange = pd.DataFrame(pctChange)*100
    pctChange = pctChange.T
    return pctChange

In [9]:
# Outstanding shares, Rev, Rev ava3, turnover, gross income/tq, cfebit adj/ tq, roic ava,
#gross margin, ebitda margin, net inc margin, cfo margin, sfcf margin, ncf margin, revs/sh,
#assets/sh, net excess cash, net com over, book/sh, tang book/sh, bm roe 3yr %, bm roe 5yr %,
# tbm ROE 3yr %, tbm roe 5yr %, div/sh, bm EPS ava, EBITDA/sh, 

In [13]:
#ticker = yf.Tickers('tsm nvda asml intc avgo qcom amd lrcx mu toelf adi nxpi mrvl klac snps xlnx swks umc qrvo asx on entg powi amba xper himx acls cohu veco sgh ceva plab aosl axti pxlw aehr quik')
#ticker = yf.Tickers('splk ddog okta u panw intu coin pltr msft')
#ticker = yf.Tickers('dkng penn mgm lvs sgms evri gan baly byd')

stonk = pd.DataFrame()
for symbol in ticker.tickers:
    dateSymbol = symbol
    symbol = ticker.tickers[symbol]
    dfList = [symbol.info['shortName'],
              symbol.info['symbol'],
              symbol.info['sector'],
              symbol.info['currentPrice'],
              (symbol.info['currentPrice'] - symbol.info['open'])
              /symbol.info['open']*100,
              symbol.info['52WeekChange'],
              symbol.info['marketCap'] / 1e9,
              symbol.info['totalRevenue'] / 1e9,
              #symbol.info['shortPercentOfFloat']*100,
              symbol.info['forwardPE'],
              symbol.info['pegRatio'],
              symbol.info['revenueGrowth']*100,
              symbol.info['trailingEps'],
              symbol.info['forwardEps'],
              symbol.info['priceToSalesTrailing12Months'],
              symbol.info['enterpriseToEbitda'],
              symbol.info['totalCashPerShare'],
              one_week_insider(dateSymbol) / 1e6,
              one_month_insider(dateSymbol) / 1e6,
              six_month_insider(dateSymbol) / 1e6]
    ratiodf = pd.DataFrame(dfList)
    ratiodf = ratiodf.T
    ratiodf.columns = ['Company Name', 'Ticker', 'Sector', 'Price','Day %', '52 Week Change (%)', 'Market Cap (b)', 'Revenue (b)', #'Short %',
                           'Forward PE', 'PEG', 'Revenue Growth (%)', 'EPS (TTM)', 'EPS (NTM)', 'P/S (TTM)',
                           'EV/EBITDA', 'Cash Per Share', '1w Insider Sales (m)','1m Insider Sales (m)','6m Insider Sales (m)']
    #Combine all ticker data into one dataframe
    stonk = pd.concat([stonk, ratiodf])

#Output
stonk = stonk.set_index('Company Name')
dtale.show(stonk)

In [1]:
ticker = 'MU'
api_key = ''
income_statement_annually = fa.income_statement(ticker, api_key, period="annual")
key_metrics_annually = fa.key_metrics(ticker, api_key, period="annual")
financial_ratios_annually = fa.financial_ratios(ticker, api_key, period="annual")

NameError: name 'fa' is not defined

In [12]:
km = key_metrics_annually.loc[['cashPerShare','revenuePerShare','netIncomePerShare',
                               'roic','roe','dividendYield']]
    
ins = income_statement_annually.loc[['weightedAverageShsOut','revenue']] / 1e9
filingDate = pd.DataFrame(income_statement_annually.loc['fillingDate'])
filingDate = filingDate.T
 

fr = financial_ratios_annually.loc[['grossProfitMargin','netProfitMargin']] * 100
fr2 = financial_ratios_annually.loc[['assetTurnover','operatingCashFlowPerShare',
                                    'freeCashFlowPerShare',
                                    'priceEarningsRatio','debtRatio']]

insDf = make_fundamental_sheet(ins)
kmDf = make_fundamental_sheet(km)
frDf = make_fundamental_sheet(fr)
frDf2 = make_fundamental_sheet(fr2)

#Combine the fundamental data from the separate api sheets
model = pd.concat([insDf, kmDf, frDf, frDf2])

revPctChg = make_percent_change(1,model)
model = pd.concat([model, revPctChg])
grossMarginPctChg = make_percent_change(8,model)
model = pd.concat([model, grossMarginPctChg])
model = model.reset_index(drop = True)

#Change data order so it is more intuitive for analysis
modelSheet = model.reindex([0, 1, 15, 8, 16, 9, 13, 14, 2, 3, 4, 5, 6, 11, 12, 10, 7])
modelSheet =modelSheet.rename(index={0: 'Shares Outstanding (b)', 1: 'Revenue (b)', 15: 'Revenue Change %',8: 'Gross Profit Margin %',
                                     16: 'Gross Profit Margin Change %', 9: 'Net Profit Margin %', 
                                     13: 'PE Ratio', 14: 'Debt Ratio', 2: 'Cash/Share', 
                                     3: 'Revenue/Share', 4: 'Net Income/Share', 5: 'ROIC', 6: 'ROE', 11: 'Operating Cash Flow/Share',
                                    12: 'Free Cash Flow/Share', 10: 'Asset Turnover', 7: 'Dividend Yield'})

#Output
modelSheet.style.background_gradient(axis=1, cmap = 'YlGnBu',subset=(modelSheet.index[[1,3,7,8,13,14]], modelSheet.select_dtypes(float).columns)).format('{:.02f}', modelSheet.select_dtypes(float).columns)


,2020,2019,2018,2017,2016,2015,2014,2013,2012,2011,2010,2009,2008,2007,2006,2005,2004,2003,2002,2001,2000,1999,1998,1997,1996,1995,1994,1993
Shares Outstanding (b),0.13,0.12,0.10,0.09,0.08,0.08,0.08,0.08,0.08,0.08,0.08,0.08,0.08,0.09,0.08,0.08,0.08,0.08,0.08,0.06,0.06,0.06,0.06,0.06,0.06,0.05,0.05,0.04
Revenue (b),3.58,5.30,3.59,3.15,3.03,2.84,2.59,2.92,2.90,2.74,2.46,2.37,2.42,2.44,2.35,1.41,1.14,1.16,0.66,0.52,0.29,0.17,0.15,0.11,0.06,0.06,0.05,0.04
Revenue Change %,-32.50,47.76,13.98,3.74,6.91,9.57,-11.25,0.67,5.73,11.51,3.79,-2.22,-0.56,3.63,66.48,23.83,-1.92,76.87,26.60,76.59,71.50,11.29,38.21,77.55,8.84,25.43,9.00,nan
Gross Profit Margin %,47.80,44.27,42.92,42.41,42.07,42.87,43.30,42.99,41.83,40.90,40.43,39.73,41.09,42.44,44.43,39.46,40.26,39.09,38.41,39.44,35.43,46.53,44.39,39.28,39.17,37.78,35.22,nan
Gross Profit Margin Change %,7.98,3.14,1.20,0.81,-1.87,-1.00,0.72,2.77,2.29,1.16,1.75,-3.29,-3.18,-4.48,12.59,-1.99,2.98,1.79,-2.61,11.32,-23.86,4.83,12.99,0.28,3.68,7.28,0.00,nan
Net Profit Margin %,-18.71,0.83,2.61,15.04,3.60,0.02,-9.00,-27.22,7.31,8.84,-2.42,-11.18,-6.33,6.57,13.91,8.56,6.27,4.43,4.69,4.57,4.08,3.91,4.87,2.06,8.76,8.67,6.09,nan
PE Ratio,-20.76,81.36,25.19,6.14,10.50,1647.88,-5.03,-1.12,3.90,2.98,-10.80,-1.80,-2.32,6.24,2.56,4.98,8.24,4.28,4.44,4.67,3.67,4.04,3.56,15.22,10.09,2.75,2.57,nan
Debt Ratio,0.82,0.87,0.93,1.01,1.11,1.13,0.75,0.65,0.60,0.57,0.60,0.61,0.60,0.77,0.80,0.87,0.76,0.81,0.68,0.85,0.82,0.65,0.63,0.66,0.71,0.24,0.29,nan
Cash/Share,13.83,3.78,4.94,3.06,2.77,2.96,2.66,3.75,3.41,3.06,3.16,9.13,8.83,2.04,2.00,1.60,1.09,1.35,0.73,0.63,0.39,0.16,0.11,0.37,0.10,0.15,0.11,nan
Revenue/Share,26.71,45.82,36.95,34.65,36.59,35.48,33.03,37.37,37.98,35.16,31.50,30.33,28.66,28.47,27.89,17.05,14.17,14.73,8.70,8.47,4.91,2.89,2.57,1.87,1.12,1.13,0.90,nan
